In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet

In [5]:
file_path = 'IRENA_RenewableEnergy_Statistics_2000-2022.csv'

with open(file_path, 'rb') as f:
    result = chardet.detect(f.read())

df_irena = pd.read_csv(file_path, encoding=result['encoding'])

In [8]:
file_path_1 = 'organised_Gen.csv'

with open(file_path_1, 'rb') as f:
    result = chardet.detect(f.read())

df_us_data = pd.read_csv(file_path_1, encoding=result['encoding'])

In [9]:
file_path_2 = '02 modern-renewable-energy-consumption.csv'

with open(file_path_2, 'rb') as f:
    result = chardet.detect(f.read())

df_world_data = pd.read_csv(file_path_2, encoding=result['encoding'])

In [17]:
df_irena_us = df_irena[df_irena["Country"] == "United States of America (the)"]

df_pivot = df_irena_us.pivot_table(
    index="Year",
    columns="Technology",
    values="Electricity Generation (GWh)",
    aggfunc="sum"
).reset_index()

df_pivot


Technology,Year,Biogas,Coal and peat,Geothermal energy,Liquid biofuels,Marine energy,Natural gas,Nuclear,Offshore wind energy,Oil,Onshore wind energy,Other non-renewable energy,Pumped storage,Renewable hydropower,Renewable municipal waste,Solar photovoltaic,Solar thermal energy,Solid biofuels
0,2000,5230.001,2129498.0,14621.0,NaN,NaN,634290.0,797718.0,NaN,118482.0,5650.000,15533.0,26782.0,253204.0,8364.0,183.0,526.0,42586.000
1,2001,5413.999,1982120.0,14246.0,NaN,NaN,659914.0,792604.0,NaN,129557.0,6806.000,14210.0,26476.0,188252.0,8102.0,220.0,565.0,36821.000
2,2002,5561.000,2039665.0,14939.0,NaN,NaN,712432.0,804519.0,NaN,106138.0,10459.000,15508.0,24721.0,267034.0,8252.0,261.0,569.0,40209.998
3,2003,6317.000,2083326.0,14870.0,NaN,NaN,670192.0,787818.0,NaN,137015.0,11300.000,15865.0,27115.0,278609.0,8093.0,300.0,548.0,39710.998
4,2004,6412.000,2090495.0,15487.0,NaN,NaN,731552.0,813339.0,NaN,139089.0,14291.000,15618.0,26776.0,271118.0,8939.0,378.0,587.0,40432.001
5,2005,6448.999,2153956.0,16778.0,213.0,NaN,782829.0,810726.0,NaN,141290.0,17881.000,13881.0,25481.0,272445.0,9528.0,524.0,596.0,41790.999
6,2006,7057.999,2127796.0,16581.0,164.0,NaN,842774.0,816195.0,NaN,78621.0,26676.000,14416.0,25820.0,291869.0,9690.0,737.0,550.0,41884.001
7,2007,7468.999,2118455.0,16798.0,84.0,NaN,915196.0,836634.0,NaN,78136.0,34603.000,13679.0,25926.0,249619.0,9578.0,1001.0,673.0,41989.999
8,2008,8427.000,2132596.0,16873.0,88.0,NaN,910176.0,837804.0,NaN,57776.0,55696.000,13606.0,25281.0,256714.0,9513.0,1214.0,879.0,41619.001
9,2009,9281.000,1892661.0,17046.0,91.0,NaN,949776.0,830210.0,NaN,50445.0,74226.000,13607.0,22820.0,275590.0,9469.0,1699.0,816.0,40477.998


In [20]:
import statsmodels.api as sm

df_pivot["Total"] = df_pivot.sum(axis=1, numeric_only=True)

df_pivot["Solar_Wind"] = df_pivot["Solar photovoltaic"] * df_pivot["Onshore wind energy"]
df_pivot["Solar2"] = df_pivot["Solar photovoltaic"] ** 2
df_pivot["Wind2"] = df_pivot["Onshore wind energy"] ** 2

X = df_pivot[["Solar photovoltaic", "Onshore wind energy", "Solar_Wind", "Solar2", "Wind2"]]
y = df_pivot["Total"]

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

print(model.summary())
print(model.params)

                            OLS Regression Results                            
Dep. Variable:                  Total   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.483e+12
Date:                Mon, 30 Jun 2025   Prob (F-statistic):           2.64e-98
Time:                        16:15:25   Log-Likelihood:                -317.94
No. Observations:                  23   AIC:                             647.9
Df Residuals:                      17   BIC:                             654.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 1.21e+07   1

In [23]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

               feature           VIF
0                const      6.568369
1   Solar photovoltaic   1824.749843
2  Onshore wind energy    216.363406
3           Solar_Wind  14992.719724
4               Solar2   4596.234298
5                Wind2   1611.304867
